# Дорогой дневник

--------------------------------------------

1) Сначала я решил попробовать без генерации новых фичей и без подкрутки параметров запустить LightGBM и посмотреть, что будет. Скор 0.720
При этом пока еще нет ни генерации новых признаков, ни использования таблицы поиска и тд. Попробуем добавить.

2) Просто запускаем код из "baseline_1_pandas.ipynb" и получаем обещаный скор 0.817

3) Оптюнил 10 минут, безрезультатно. Буду придумывать новые признаки. Думаю начать с кластеризации и knn. 

4) Сначала решил просто расширить плавающее окно с 4 до 5 месяцев, результат 0.8192.

    Есть огромное количество идей, только что заменил пандас на поларс, потому что он реально на порядок быстрее. В первую очередь хочется применить знания с семинара по интерпретации бустингов, но сначала заменить катбуст на lgbm, ибо Илья утверждал, что при должном обращении он рвет и мечет.

5) Провел маштабное расследование в данных. Сначала просто обучил lgbm со скором примерно 0.817 и применил на нам всю информацию из семинара про интерпретацию бустингов. Пока что не совсем понятно как это оформиать и как интерпретировать некотрые вещи, но что точно ясно это то, что модель досточно устойчива и надежна, она не реазирует на теневые фичи, а это значит что местами можно делать бред не боясь за последствия. Например сделать полиномиальные фичи - всео со всеми. 

6) Это я и сделал, вмете с новыми фиками добавил количество поисков на последний месяц и неделю. Паралельно с этим убрад один выбрас user_id = 2346229. У человека были абсолютно неадекватные значения. Есть еще большое количество таких людей, но у них значения не такие радикальные и я боюсь что они могут оказаться просто очень активными пользователями, выкидывать их значит лишать себя бесценной информации. После добавления пачуи полиномиальных фичей скор стал 0.8210. Дальше планирую применить кластеризацию на поиске и заказах. Мне кажется очень логичным, что если человек совершал покупки одной категории, то он вероятнее купит еще, чем другой человек с та4ким же числом покупок, но разных категорий. 

7) Кластеризация. Очень долго ей занимался, сейчас подведу итоги всех трех дней. Главная идея такая - модель уже использует агригаты, так что для того чтобы повысить качество нужно использовать какую то иную информацию. Первое что проходит на ум - запросы человека. Банально провести кластеризацию на запросах и потом построить на этих метках новые агрегаты. Но есть проблема, у нас 77кк строк запросов и даже если написать нормальный алгоритм это будет работать 19 часов. Помимо того на сэмпле и понижении размерности через PCA выяснилось, что данные это полосы, крупные классы запиывают мелкие и те забиваются одной неразделимой кучей в углу, не получается чегото адекватного даже если использовать гауссовскую кластеризациюд или GMM. 300к обьектов обрабатывались несколько минут и я решил сделать кластеризацию полуавтоматически. Я создал столбец 'cluster' и заполнил его -1. Построив кластеризацию на 10_000 обьектах выделил некоторые классы, они ясно выделаются, например 20-30 обьектов с словом "вода" в названии, и заполнил их метки руками. Потом снова построил кластеризацию на оставшихся. Леноточная структура осталась, но обьекты былит уже другими. за каждую такую итерацию отсеивалось от 3кк до 10кк обьектов. Итого у меня определилось 18 кластеров - напитки, питомцы, мясо и многое другое. В итоге еще осталось 23кк обьектов, но там очень сложно выделить группы. В принципе я могу свести их до 10кк, но вопрос есть ли в этом смысл. После таких не простых манипуляций мы получаем метки, но это просто инструмент. То что реально даст прирост качества это агрегаты. Не просто было придумать их и не все оказались удачными. Я добавил последний доминирующий кластер, концентрация интересов, переключения между кластерами, стабильность кластеров, время в основном кластере, среднее время между сменами кластеров, средняя длина запроса, разница длины запросов. В итоге скор получился 0.8221.

--------------------------------------------

In [ ]:
import polars as pl
import pandas as pd
import numpy as np
import catboost
import os
from datetime import date, timedelta
from itertools import combinations

from local_utils import *
import lightgbm as lgb

%load_ext autoreload
%autoreload 2

test_start_date = date(2024, 8, 1)
val_start_date = date(2024, 7, 1)
val_end_date = date(2024, 7, 31)
train_end_date = date(2024, 6, 30)
data_path = "C:\\Users\\Admin\\Desktop\\AIM 2сем\\ML2\\hw2"

actions_history = pl.scan_parquet(os.path.join(data_path, 'actions_history/*.parquet')).collect()
search_history = pl.scan_parquet(os.path.join(data_path, 'search_history/*.parquet')).collect()
product_information = pl.read_csv(
    os.path.join(data_path, 'product_information.csv'),
    ignore_errors=True
)

val_target = (
    actions_history
    .filter(pl.col('timestamp').dt.date() >= val_start_date)
    .filter(pl.col('timestamp').dt.date() <= val_end_date)
    .select('user_id', (pl.col('action_type_id') == 3).alias('has_order'))
    .group_by('user_id')
    .agg(pl.max('has_order').cast(pl.Int32).alias('target'))
)

val_target.group_by('target').agg(pl.count('user_id'))

sh_topd = search_history.to_pandas()
sh_topd

sh_topd = sh_topd.dropna(subset=['search_query'])

In [ ]:
sh_topd['cluster'] = -1

In [ ]:
milk = ['кефир', 'сметан', 'масло', 'йогурт', 'морожен', 'творог', 'яйц', 'молок', 'сливк', 'ряжен', 'простокваш', 'пахт', 'сгущенк', 'сыворот']

sweet = ['конфет', 'мармелад', 'сироп', 'энергетик', 'кекс', 'сахар', 'шоколад', 'батончик', 'варен', 'печень', 'торт', 'чипс', 'лейс', 'lays', 'халв', 
         'зефир', 'пастил', 'пряник', 'леден', 'драже', 'карамел', 'джем', 'мед', 'вафл', 'хлебец', 'семечки']

cereals = ["греч", "рис", "овсян", "пшен", "перлов", "манк", "кукурузн", "ячнев", "булгур", "киноа", "макарон", "спагетт", "лапш", "вермишел", "фасол", 
           "горох", "нут", "чечевиц", "полб", 'каша']

veg_and_fruits = ['изюм', 'перец', 'огур', 'помидор', 'шампиньон', 'банан', 'фрукт', 'овощ', 'апельсин', 
                  'томат', 'свекл', 'баклажан', 'мандарин', 'ягод', 'гриб', 'лимон', 'персик', 'цукин', 
                  'яблок', 'груш', 'салат', 'черри', 'морков', 'патиссон', 'картош', 'картоф','капуст',
                  'груша', 'персик', 'апельсин', 'мандарин', 'баклажан', 'лук', 'авокадо', 'клубника', 'зелен', 'кукуруз'
]

drink = ['квас', 'сок', 'вод', 'кофе', 'чай', 'лимонад', 'напитк', 'кола']

bread = ['хлеб', 'булк', 'батон', 'багет', 'еда', 'мука', 'пицца', 'лаваш']

cheese = ['сыр', 'брынз', 'моцарел']

pets = ['корм', 'наполнит', 'лакомств', 'вискас', 'педигри', 'собак', 'кошек', 'кроликов']

meat = ['рыб', 'сосис', 'колбас', 'куриц', 'филе', 'говядин', 'мяс', 'крабов', 'креветк', 'свинин', 'фарш', 'кальмар', 'ветчина', 'пельмени', 'бекон'
        'индейка', 'котлет', 'суп', 'икр', 'груд', ]

household = ['мыл', 'гел', 'паст', 'туалетн', 'крем', 'средств', 'краск', 'стакан', 'волос', 'шампун', 'кондиционер', 'салфетк', 'порошок', 'жидкост', 
             'капсул', 'бумаг', 'бумаж', 'полотен', 'проклад']

kids = ['подгузник', 'памперс', 'дет', 'соск', 'пюре', ]

clothing = [
    'куртк', 'пальт', 'пуховик', 'ветровк', 'плащ', 'шуб', 'дубленк', 'зимн', 'демисезонн',
    'футболк', 'рубашк', 'блузк', 'водолазк', 'толстовк', 'свитер', 'джемпер', 'кардиган', 'худи',
    'кофт', 'боди', 'топ', 'майк',
    'брюк', 'джинс', 'штан', 'штаны', 'леггинс', 'бридж', 'капри', 'юбк', 'сарафан', 'комбинезон',
    'плать', 'сарафан', 'туник', 'сарафан',
    'трусы', 'трусики', 'бюстгальтер', 'лифчик', 'белье', 'пижам', 'ночн', 'халат', 'комплект',
    'спортивн', 'лосин', 'велосипедк', 'термобелье', 'олимпийк', 'тренировочн', 'спорткостюм',
    'купальник', 'бикини', 'плавк', 'монокини', 'танкини', 'пляжн',
    'шапк', 'кепк', 'шляп', 'панам', 'косынк', 'бандан', 'берет',
    'туфл', 'кроссовк', 'ботинк', 'сапог', 'босоножк', 'сандал', 'угг', 'каблук', 'мокасин', 'лофер',
    'черепик', 'кед', 'балетк', 'сабо', 'дутик',
    'ремен', 'пояс', 'галстук', 'бабочк', 'шарф', 'перчатк', 'варежк', 'зонт', 'очк', 'солнцезащитн',
    'сумк', 'рюкзак', 'клатч', 'кошельк', 'портфель',
    'халат', 'униформа', 'спецовк', 'комбинезон', 'фартук',
    'детск', 'малыш', 'подросток', 'распашонк', 'ползунк', 'пеленк',
    'носок', 'гольф', 'чулк', 'колготк', 'подследник',
    'аксессуар', 'декор', 'украшен', 'бижутери', 'стильн', 'модн', 'брендов'
]

health = [
    # Органы и системы
    'глаз', 'зрен', 'офтальм', 'зрит', 'сетчатк', 'роговиц',
    'сердц', 'карди', 'гипертон', 'давлен', 'пульс', 'аритм',
    'печен', 'гепат', 'желч', 'желуд', 'гастр', 'кишечн', 'панкреат',
    'почк', 'нефр', 'мочевой', 'цистит', 'простат',
    'легк', 'пневм', 'бронх', 'астм', 'кашел', 'насморк', 'горл', 'отит',
    'зуб', 'стомат', 'десн', 'кариес', 'флюс',
    'кож', 'дермат', 'акне', 'псориаз', 'экзем', 'аллерг',
    'нерв', 'неврол', 'головн', 'мигрен', 'инсульт', 'эпилепс',
    'сустав', 'артр', 'остеохондр', 'позвон', 'ревмат', 'артрит',
    'щитовид', 'эндокрин', 'диабет', 'гормон', 'инсулин',
    
    # Препараты и формы выпуска
    'таблетк', 'капсул', 'капл', 'маз', 'крем', 'гел', 'спрей', 'ингалятор',
    'суспенз', 'сироп', 'настойк', 'экстракт', 'назол', 'отинум',
    'антибиот', 'противовирусн', 'антигистаминн', 'анальгин', 'парацетамол',
    'ибупрофен', 'спазмолгон', 'но-шп', 'корвалол', 'валидол',
    'витамин', 'минерал', 'омега', 'магн', 'коллаген', 'пробиот',
    
    # Симптомы и состояния
    'бол', 'воспален', 'отек', 'опухол', 'зуд', 'жжен', 'тошнот', 'рвот',
    'диаре', 'запо', 'метеоризм', 'слабост', 'усталост', 'стресс', 'депресс',
    'бессонн', 'тревожн', 'паническ', 'судорог', 'свист', 'хрип',
    'кровоточивост', 'кров', 'гемоглоб', 'анеми', 'варикоз', 'геморро',
]

sauces = [
    'майонез', 'кетчуп', 'горчиц', 'аджик', 'хрен', 'соус', 'тартар', 'бешамель', 'песто', 'цезар','барбекю', 'чили', 'терияк', 'сальса', 'гуакамоле',
    'сметанн', 'чесночн', 'имбирн', 'устричн', 'рыбн','ткемал', 'кисло-сладк', 'бальзамик', 'вустершир', 'карри','сырн', 'шашлычн', 'гриль', 'паприк', 'чеддер',
    'провансаль', 'брусничн', 'клюквен', 'мятн', 'укропн'
]

gadgets = [
    'смартфон', 'iphone', 'андроид', 'телефон', 'кнопочный телефон', 'раскладушка',
    'xiaomi', 'samsung', 'honor', 'huawei', 'nokia', 'realme', 'oppo', 'vivo',
    'планшет', 'ipad', 'электронная книга', 'электронная читалка', 'букридер',
    'ноутбук', 'ультрабук', 'macbook', 'asus', 'lenovo', 'hp', 'dell', 'acer', 'msi',
    'компьютер', 'системный блок', 'моноблок', 'неттоп',
    'клавиатура', 'мышь', 'мышка', 'коврик для мыши', 'трекпад', 'стилус', 'графический планшет',
    'веб-камера', 'микрофон', 'док-станция',
    'процессор', 'видеокарта', 'оперативная память', 'ssd', 'жесткий диск', 'hdd',
    'материнская плата', 'блок питания', 'охлаждение', 'кулер', 'термопаста',
    'наушники', 'airpods', 'гарнитура', 'беспроводные наушники', 'накладные наушники',
    'вкладыши', 'вакуумные', 'колонка', 'динамик', 'bluetooth колонка', 'портативная колонка',
    'микрофон', 'звуковая карта', 'dac', 'усилитель',
    'умные часы', 'smart watch', 'фитнес-браслет', 'умный браслет', 'пульсометр',
    'шагомер', 'gps трекер', 'умное кольцо',
    'чехол', 'защитное стекло', 'плёночка', 'бампер', 'держатель', 'попсокет',
    'кардхолдер', 'кабель', 'зарядка', 'адаптер', 'power bank', 'повербанк',
    'кабель usb', 'кабель type-c', 'lightning', 'переходник', 'разветвитель',
    'подставка', 'держатель для телефона', 'автодержатель',
    'игровая мышь', 'игровая клавиатура', 'джойстик', 'геймпад', 'руль', 'педали',
    'аркадный стик', 'vr', 'виртуальная реальность', 'очки vr', 'гарнитура vr',
    'фотоаппарат', 'зеркалка', 'беззеркалка', 'экшн-камера', 'go pro', 'объектив',
    'штатив', 'монопод', 'стабилизатор', 'гимбал', 'квадрокоптер', 'дрон',
    'умная лампа', 'умная розетка', 'умный дом', 'датчик', 'сенсор', 'камера наблюдения',
    'видеоняня', 'умный замок', 'робот-пылесос', 'умный термостат',
    'принтер', 'сканер', 'ксерокс', 'мфу', 'факс', 'ламинтор', 'брошюровщик',
    'уничтожитель бумаг', 'шредер',
    'электроника', 'гаджет', 'новинка', 'аксессуар', 'комплектующая', 'устройство',
    'беспроводной', 'bluetooth', 'wi-fi', 'usb', 'type-c', 'lightning', 'адаптер'
]

spices = [
    'перец', 'соль', 'паприк', 'куркум', 'кориандр', 'кардамон', 'имбир', 
    'чеснок', 'лук', 'лавров', 'розмарин', 'тимьян', 'базилик', 'орегано', 
    'шалфей', 'мята', 'укроп', 'петрушк', 'кинз', 'тмин', 'зир', 'кориц', 
    'гвоздик', 'мускат', 'ванил', 'шафран', 'горчиц', 'хрен', 'карри', 
    'прованс', 'фенхел', 'анис', 'кумин', 'чили', 'бадьян', 'кайен', 'тархун'
]

In [ ]:
sh_topd.loc[sh_topd['search_query'].str.contains('|'.join(milk), case=False, na=False), 'cluster'] = 1
sh_topd.loc[sh_topd['search_query'].str.contains('|'.join(sweet), case=False, na=False), 'cluster'] = 2
sh_topd.loc[sh_topd['search_query'].str.contains('|'.join(cereals), case=False, na=False), 'cluster'] = 3
sh_topd.loc[sh_topd['search_query'].str.contains('|'.join(veg_and_fruits), case=False, na=False), 'cluster'] = 4
sh_topd.loc[sh_topd['search_query'].str.contains('|'.join(drink), case=False, na=False), 'cluster'] = 5
sh_topd.loc[sh_topd['search_query'].str.contains('|'.join(bread), case=False, na=False), 'cluster'] = 6
sh_topd.loc[sh_topd['search_query'].str.contains('|'.join(cheese), case=False, na=False), 'cluster'] = 7
sh_topd.loc[sh_topd['search_query'].str.contains('|'.join(pets), case=False, na=False), 'cluster'] = 8
sh_topd.loc[sh_topd['search_query'].str.contains('|'.join(meat), case=False, na=False), 'cluster'] = 9
sh_topd.loc[sh_topd['search_query'].str.contains('|'.join(household), case=False, na=False), 'cluster'] = 10
sh_topd.loc[sh_topd['search_query'].str.contains('|'.join(kids), case=False, na=False), 'cluster'] = 11
sh_topd.loc[sh_topd['search_query'].str.contains('|'.join(clothing), case=False, na=False), 'cluster'] = 12
sh_topd.loc[sh_topd['search_query'].str.contains('|'.join(health), case=False, na=False), 'cluster'] = 13
sh_topd.loc[sh_topd['search_query'].str.contains('|'.join(sauces), case=False, na=False), 'cluster'] = 14
sh_topd.loc[sh_topd['search_query'].str.contains('|'.join(gadgets), case=False, na=False), 'cluster'] = 15
sh_topd.loc[sh_topd['search_query'].str.contains('для', case=False, na=False), 'cluster'] = 16
sh_topd.loc[sh_topd['search_query'].str.contains('цвет', case=False, na=False), 'cluster'] = 17
sh_topd.loc[sh_topd['search_query'].str.contains('|'.join(spices), case=False, na=False), 'cluster'] = 18


remaining_negatives = (sh_topd['cluster'] == -1).sum()
print(f"Осталось строк с меткой -1: {remaining_negatives}")

In [ ]:
sh_topd_filtered = sh_topd[sh_topd['cluster'] == -1].copy()
cluster_search_queries(sh_topd_filtered, 'search_query', 100_000)

--------------------------------------------

In [ ]:
import polars as pl
import pandas as pd
import numpy as np
import catboost
import os
from datetime import date, timedelta
from itertools import combinations

from local_utils import *
import lightgbm as lgb

%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
test_start_date = date(2024, 8, 1)
val_start_date = date(2024, 7, 1)
val_end_date = date(2024, 7, 31)
train_end_date = date(2024, 6, 30)
data_path = "C:\\Users\\Admin\\Desktop\\AIM 2сем\\ML2\\hw2"

In [ ]:
actions_history = pl.scan_parquet(os.path.join(data_path, 'actions_history/*.parquet')).collect()
search_history = pl.scan_parquet(os.path.join(data_path, 'search_history/*.parquet')).collect()
product_information = pl.read_csv(
    os.path.join(data_path, 'product_information.csv'),
    ignore_errors=True
)

In [ ]:
val_target = (
    actions_history
    .filter(pl.col('timestamp').dt.date() >= val_start_date)
    .filter(pl.col('timestamp').dt.date() <= val_end_date)
    .select('user_id', (pl.col('action_type_id') == 3).alias('has_order'))
    .group_by('user_id')
    .agg(pl.max('has_order').cast(pl.Int32).alias('target'))
)

In [ ]:
val_target.group_by('target').agg(pl.count('user_id'))

target,user_id
i32,u32
0,1227381
1,647575


In [ ]:
actions_aggs = {}
actions_id_to_suf = {
    1: "click",
    2: "favorite", 
    3: "order",
    5: "to_cart",
}

# Сначала соберем все агрегированные данные
all_aggs = []
numeric_features = []

for id_, suf in actions_id_to_suf.items():
    aggs = (
        actions_history
        .filter(pl.col('timestamp').dt.date() <= train_end_date)
        .filter(pl.col('timestamp').dt.date() >= train_end_date - timedelta(days=30 * 4))
        .filter(pl.col('action_type_id') == id_)
        .join(
            product_information.select('product_id', 'discount_price'),
            on='product_id',
        )
        .group_by('user_id')
        .agg(
            pl.count('product_id').cast(pl.Int32).alias(f'num_products_{suf}'),
            pl.sum('discount_price').cast(pl.Float32).alias(f'sum_discount_price_{suf}'),
            pl.max('discount_price').cast(pl.Float32).alias(f'max_discount_price_{suf}'),
            pl.max('timestamp').alias(f'last_{suf}_time'),
            pl.min('timestamp').alias(f'first_{suf}_time'),
        )
        .with_columns([
            (pl.lit(val_start_date) - pl.col(f'last_{suf}_time'))
            .dt.total_days()
            .cast(pl.Int32)
            .alias(f'days_since_last_{suf}'),
            
            (pl.lit(val_start_date) - pl.col(f'first_{suf}_time'))
            .dt.total_days()
            .cast(pl.Int32)
            .alias(f'days_since_first_{suf}'),
        ])
    )
    
    # Сохраняем имена числовых фичей для последующего создания полиномов
    numeric_features.extend([
        f'num_products_{suf}',
        f'sum_discount_price_{suf}', 
        f'max_discount_price_{suf}',
        f'days_since_last_{suf}',
        f'days_since_first_{suf}',
    ])
    
    actions_aggs[id_] = aggs
    all_aggs.append(aggs)

# Объединяем все агрегации по user_id с указанием суффиксов
combined = all_aggs[0]
for i, agg in enumerate(all_aggs[1:], 1):
    combined = combined.join(
        agg, 
        on='user_id', 
        how='left',
        suffix=f"_{i}"  # Добавляем уникальный суффикс для каждого соединения
    )

In [ ]:
if 1 and (not isinstance(sh_topd, pl.DataFrame)):
    sh_topd = pl.from_pandas(sh_topd)  # Конвертируем pandas -> polars

    # Ключи для объединения
    join_keys = ['user_id', 'timestamp', 'search_query']

    # Выполняем left join
    search_history = search_history.join(
        sh_topd.select(join_keys + ['cluster']),  # Выбираем нужные колонки
        on=join_keys,
        how='left'
    )
    
# search_aggs
id_ = 4
suf = 'search'

# Сначала вычислим value_counts отдельно и развернем их в плоскую структуру
cluster_counts = (
    search_history
    .filter(pl.col('action_type_id') == id_)
    .filter(pl.col('timestamp').dt.date() <= train_end_date)
    .filter(pl.col('timestamp').dt.date() >= train_end_date - timedelta(days=30 * 5))
    .group_by('user_id')
    .agg(
        pl.col('cluster').value_counts().alias('cluster_counts')
    )
    .explode('cluster_counts')
    .with_columns(
        pl.col('cluster_counts').struct.field('cluster').alias('cluster_name'),
        pl.col('cluster_counts').struct.field('count').alias('cluster_count')
    )
    .group_by('user_id')
    .agg(
        pl.col('cluster_name').sort_by('cluster_count', descending=True).head(3).alias('top3_clusters'),
        pl.col('cluster_count').sort(descending=True).head(3).alias('top3_counts')
    )
)

actions_aggs[id_] = (
    search_history
    .filter(pl.col('action_type_id') == id_)
    .filter(pl.col('timestamp').dt.date() <= train_end_date)
    .filter(pl.col('timestamp').dt.date() >= train_end_date - timedelta(days=30 * 5))
    .group_by('user_id')
    .agg(
        # Общее количество поисков за 5 месяцев
        pl.count('search_query').cast(pl.Int32).alias(f'num_{suf}'),
        pl.col('search_query').n_unique().alias(f'unique_{suf}_queries'),
        
        # Количество поисков за последний месяц (30 дней)
        pl.col('search_query')
            .filter(pl.col('timestamp').dt.date() >= train_end_date - timedelta(days=30))
            .count()
            .cast(pl.Int32)
            .alias(f'num_{suf}_last_month'),
        
        # Количество поисков за последнюю неделю (7 дней)
        pl.col('search_query')
            .filter(pl.col('timestamp').dt.date() >= train_end_date - timedelta(days=7))
            .count()
            .cast(pl.Int32)
            .alias(f'num_{suf}_last_week'),

        (pl.count() / (pl.max('timestamp') - pl.min('timestamp')).dt.total_days()).alias(f'{suf}_daily_rate'),

        pl.col('cluster').n_unique().alias(f'num_{suf}_clusters'),
        pl.col('cluster').mode().first().alias(f'main_{suf}_cluster'),
        
        # Динамика кластеров
        pl.col('cluster')
            .filter(pl.col('timestamp').dt.date() >= train_end_date - timedelta(days=30))
            .mode().first()
            .alias(f'recent_{suf}_cluster'),

        (pl.col('cluster').value_counts().struct.field('count').max() / pl.col('cluster').count()).alias(f'{suf}_cluster_concentration'),
        
        # Энтропия кластеров (мера разнообразия)
        (-(pl.col('cluster').value_counts().struct.field('count') / pl.col('cluster').count()).log()
            * (pl.col('cluster').value_counts().struct.field('count') / pl.col('cluster').count())
            .sum()).alias(f'{suf}_cluster_entropy'),
        
        # Переключения между кластерами
        pl.col('cluster').diff().fill_null(0).abs().sum().alias(f'{suf}_cluster_switches'),
        
        # Стабильность кластеров (процент повторяющихся)
        ((pl.col('cluster').count() - pl.col('cluster').n_unique()) / pl.col('cluster').count())
            .alias(f'{suf}_cluster_stability'),
        
        # Время в основном кластере
        (pl.col('timestamp')
            .filter(pl.col('cluster') == pl.col('cluster').mode().first())
            .count() / pl.col('timestamp').count())
            .alias(f'main_{suf}_cluster_time_ratio'),

        pl.col('timestamp').filter(pl.col('cluster').diff().fill_null(0) != 0)
            .diff()
            .dt.total_days()
            .mean()
            .alias(f'{suf}_mean_cluster_switch_days'),

        pl.col('search_query').str.len_chars().mean().alias(f'{suf}_mean_query_len'),
        
        (pl.col('search_query').str.len_chars()
            .filter(pl.col('cluster') == pl.col('cluster').mode().first()).mean() - 
            pl.col('search_query').str.len_chars()
                .filter(pl.col('cluster') != pl.col('cluster').mode().first()).mean())
                .alias(f'{suf}_main_cluster_query_len_diff'),

        pl.max('timestamp').alias(f'last_{suf}_time'),
        pl.min('timestamp').alias(f'first_{suf}_time'),
    )
    .join(cluster_counts, on='user_id', how='left')
    .with_columns([
        (pl.lit(val_start_date) - pl.col(f'last_{suf}_time'))
            .dt.total_days()
            .cast(pl.Int32)
            .alias(f'days_since_last_{suf}'),

        (pl.lit(val_start_date) - pl.col(f'first_{suf}_time'))
            .dt.total_days()
            .cast(pl.Int32)
            .alias(f'days_since_first_{suf}'),
    ])
    .select(
        'user_id',
        f'num_{suf}',
        f'unique_{suf}_queries',
        f'num_{suf}_last_month',
        f'num_{suf}_last_week',
        f'{suf}_daily_rate',
        f'num_{suf}_clusters',
        f'main_{suf}_cluster',
        pl.col('top3_clusters').alias(f'top3_{suf}_clusters'),
        pl.col('top3_counts').alias(f'top3_{suf}_counts'),
        f'recent_{suf}_cluster',
        f'{suf}_cluster_concentration',
        f'{suf}_cluster_entropy',
        f'{suf}_cluster_switches',
        f'{suf}_cluster_stability',
        f'main_{suf}_cluster_time_ratio',
        f'{suf}_mean_cluster_switch_days',
        f'{suf}_mean_query_len',
        f'{suf}_main_cluster_query_len_diff',
        f'days_since_last_{suf}',
        f'days_since_first_{suf}',
        f'last_{suf}_time',
        f'first_{suf}_time',
    )
)

In [ ]:
df_main = val_target
for _, actions_aggs_df in actions_aggs.items():
    df_main = (
        df_main
        .join(actions_aggs_df, on='user_id', how='left')
    )
    
df_pd = df_main.to_pandas()

num_cols = ['num_products_click', 'sum_discount_price_click', 'max_discount_price_click', 'days_since_last_click', 'days_since_first_click', 
    'num_products_favorite', 'sum_discount_price_favorite', 'max_discount_price_favorite', 'days_since_last_favorite', 
    'days_since_first_favorite', 'num_products_order', 'sum_discount_price_order', 'max_discount_price_order', 'days_since_last_order', 
    'days_since_first_order', 'num_products_to_cart', 'sum_discount_price_to_cart', 'max_discount_price_to_cart', 'days_since_last_to_cart', 
    'days_since_first_to_cart']

In [ ]:
from local_utils import *

df_pd = add_polynomial_features_pd(df_pd, num_cols, degree=3)

df_pd = df_pd[df_pd['user_id'] != 2346229]

tr, val = get_split(df_pd, val_size=0.3)

cols = [
    # Существующие столбцы
    'num_products_click', 
    'sum_discount_price_click', 'max_discount_price_click',
    'days_since_last_click', 'days_since_first_click',
    'num_products_favorite', 'sum_discount_price_favorite',
    'max_discount_price_favorite', 'days_since_last_favorite',
    'days_since_first_favorite', 'num_products_order',
    'sum_discount_price_order', 'max_discount_price_order',
    'days_since_last_order', 'days_since_first_order',
    'num_products_to_cart', 'sum_discount_price_to_cart',
    'max_discount_price_to_cart', 'days_since_last_to_cart',
    'days_since_first_to_cart', 'num_search', 'days_since_last_search',
    'days_since_first_search', 
    'main_search_cluster', 'num_search_clusters',
    
    # Новые столбцы из анализа поиска
    'unique_search_queries',               # Количество уникальных поисковых запросов
    'num_search_last_month',               # Поисков за последний месяц
    'num_search_last_week',                # Поисков за последнюю неделю
    'search_daily_rate',                   # Среднедневная частота поисков
    'recent_search_cluster',               # Последний доминирующий кластер
    'search_cluster_concentration',        # Концентрация интересов
    'search_cluster_switches',             # Переключения между кластерами
    'search_cluster_stability',            # Стабильность кластеров
    'main_search_cluster_time_ratio',      # Время в основном кластере
    'search_mean_cluster_switch_days',     # Среднее время между сменами кластеров
    'search_mean_query_len',               # Средняя длина запроса
    'search_main_cluster_query_len_diff',  # Разница длины запросов
]

params={
        'objective': 'binary',
        'metric': 'auc',
        'learning_rate': 0.01,
        'max_depth': 100,
        'num_iterations': 700,
        'early_stopping_rounds': 60,
        'verbose': 1,
        'importance_type': 'split'
    }

model = train_model(tr, val, cols, 'target', params=params, shadow_features=False, sklearn_style=True)


In [ ]:
from local_utils import *

print('feature_importance:', model.feature_importances_, '\n')

plot_lgbm_importance(model, cols, importance_type='split', top_k=30, sklearn_style=True)

test_users_submission = (
    pl.read_csv(os.path.join(data_path, 'test_users.csv'))
)

In [ ]:
actions_aggs = {}
actions_id_to_suf = {
    1: "click",
    2: "favorite", 
    3: "order",
    5: "to_cart",
}

# Сначала соберем все агрегированные данные
all_aggs = []
numeric_features = []

for id_, suf in actions_id_to_suf.items():
    aggs = (
        actions_history
        .filter(pl.col('timestamp').dt.date() <= val_end_date)
        .filter(pl.col('timestamp').dt.date() >= val_end_date - timedelta(days=30 * 5))
        .filter(pl.col('action_type_id') == id_)
        .join(
            product_information.select('product_id', 'discount_price'),
            on='product_id',
        )
        .group_by('user_id')
        .agg(
            pl.count('product_id').cast(pl.Int32).alias(f'num_products_{suf}'),
            pl.sum('discount_price').cast(pl.Float32).alias(f'sum_discount_price_{suf}'),
            pl.max('discount_price').cast(pl.Float32).alias(f'max_discount_price_{suf}'),
            pl.max('timestamp').alias(f'last_{suf}_time'),
            pl.min('timestamp').alias(f'first_{suf}_time'),
        )
        .with_columns([
            (pl.lit(test_start_date) - pl.col(f'last_{suf}_time'))
            .dt.total_days()
            .cast(pl.Int32)
            .alias(f'days_since_last_{suf}'),
            
            (pl.lit(test_start_date) - pl.col(f'first_{suf}_time'))
            .dt.total_days()
            .cast(pl.Int32)
            .alias(f'days_since_first_{suf}'),
        ])
    )
    
    # Сохраняем имена числовых фичей для создания полиномов
    numeric_features.extend([
        f'num_products_{suf}',
        f'sum_discount_price_{suf}', 
        f'max_discount_price_{suf}',
        f'days_since_last_{suf}',
        f'days_since_first_{suf}',
    ])
    
    actions_aggs[id_] = aggs
    all_aggs.append(aggs)

# Объединяем все агрегации по user_id с суффиксами
combined_val = all_aggs[0]
for i, agg in enumerate(all_aggs[1:], 1):
    combined_val = combined_val.join(
        agg, 
        on='user_id', 
        how='outer',
        suffix=f"_{i}"
    )

In [ ]:
id_ = 4
suf = 'search'

# Вычисляем top3 кластеров для валидации (аналогично трейну)
val_cluster_counts = (
    search_history
    .filter(pl.col('action_type_id') == id_)
    .filter(pl.col('timestamp').dt.date() <= val_end_date)
    .filter(pl.col('timestamp').dt.date() >= val_end_date - timedelta(days=30 * 5))
    .group_by('user_id')
    .agg(
        pl.col('cluster').value_counts().alias('cluster_counts')
    )
    .explode('cluster_counts')
    .with_columns(
        pl.col('cluster_counts').struct.field('cluster').alias('cluster_name'),
        pl.col('cluster_counts').struct.field('count').alias('cluster_count')
    )
    .group_by('user_id')
    .agg(
        pl.col('cluster_name').sort_by('cluster_count', descending=True).head(3).alias('top3_clusters'),
        pl.col('cluster_count').sort(descending=True).head(3).alias('top3_counts')
    )
)

actions_aggs[id_] = (
    search_history
    .filter(pl.col('action_type_id') == id_)
    .filter(pl.col('timestamp').dt.date() <= val_end_date)
    .filter(pl.col('timestamp').dt.date() >= val_end_date - timedelta(days=30 * 5))
    .group_by('user_id')
    .agg(
        # Общее количество поисков за 5 месяцев
        pl.count('search_query').cast(pl.Int32).alias(f'num_{suf}'),
        pl.col('search_query').n_unique().alias(f'unique_{suf}_queries'),
        
        # Количество поисков за последний месяц (30 дней)
        pl.col('search_query')
            .filter(pl.col('timestamp').dt.date() >= val_end_date - timedelta(days=30))
            .count()
            .cast(pl.Int32)
            .alias(f'num_{suf}_last_month'),
        
        # Количество поисков за последнюю неделю (7 дней)
        pl.col('search_query')
            .filter(pl.col('timestamp').dt.date() >= val_end_date - timedelta(days=7))
            .count()
            .cast(pl.Int32)
            .alias(f'num_{suf}_last_week'),

        (pl.count() / (pl.max('timestamp') - pl.min('timestamp')).dt.total_days()).alias(f'{suf}_daily_rate'),

        pl.col('cluster').n_unique().alias(f'num_{suf}_clusters'),
        pl.col('cluster').mode().first().alias(f'main_{suf}_cluster'),
        
        # Динамика кластеров
        pl.col('cluster')
            .filter(pl.col('timestamp').dt.date() >= val_end_date - timedelta(days=30))
            .mode().first()
            .alias(f'recent_{suf}_cluster'),

        (pl.col('cluster').value_counts().struct.field('count').max() / pl.col('cluster').count()).alias(f'{suf}_cluster_concentration'),
        
        # Энтропия кластеров
        (-(pl.col('cluster').value_counts().struct.field('count') / pl.col('cluster').count()).log()
            * (pl.col('cluster').value_counts().struct.field('count') / pl.col('cluster').count())
            .sum()).alias(f'{suf}_cluster_entropy'),
        
        # Переключения между кластерами
        pl.col('cluster').diff().fill_null(0).abs().sum().alias(f'{suf}_cluster_switches'),
        
        # Стабильность кластеров
        ((pl.col('cluster').count() - pl.col('cluster').n_unique()) / pl.col('cluster').count())
            .alias(f'{suf}_cluster_stability'),
        
        # Время в основном кластере
        (pl.col('timestamp')
            .filter(pl.col('cluster') == pl.col('cluster').mode().first())
            .count() / pl.col('timestamp').count())
            .alias(f'main_{suf}_cluster_time_ratio'),

        pl.col('timestamp').filter(pl.col('cluster').diff().fill_null(0) != 0)
            .diff()
            .dt.total_days()
            .mean()
            .alias(f'{suf}_mean_cluster_switch_days'),

        pl.col('search_query').str.len_chars().mean().alias(f'{suf}_mean_query_len'),
        
        (pl.col('search_query').str.len_chars()
            .filter(pl.col('cluster') == pl.col('cluster').mode().first()).mean() - 
            pl.col('search_query').str.len_chars()
                .filter(pl.col('cluster') != pl.col('cluster').mode().first()).mean())
                .alias(f'{suf}_main_cluster_query_len_diff'),

        pl.max('timestamp').alias(f'last_{suf}_time'),
        pl.min('timestamp').alias(f'first_{suf}_time'),
    )
    .join(val_cluster_counts, on='user_id', how='left')
    .with_columns([
        (pl.lit(test_start_date) - pl.col(f'last_{suf}_time'))
            .dt.total_days()
            .cast(pl.Int32)
            .alias(f'days_since_last_{suf}'),

        (pl.lit(test_start_date) - pl.col(f'first_{suf}_time'))
            .dt.total_days()
            .cast(pl.Int32)
            .alias(f'days_since_first_{suf}'),
    ])
    .select(
        'user_id',
        f'num_{suf}',
        f'unique_{suf}_queries',
        f'num_{suf}_last_month',
        f'num_{suf}_last_week',
        f'{suf}_daily_rate',
        f'num_{suf}_clusters',
        f'main_{suf}_cluster',
        pl.col('top3_clusters').alias(f'top3_{suf}_clusters'),
        pl.col('top3_counts').alias(f'top3_{suf}_counts'),
        f'recent_{suf}_cluster',
        f'{suf}_cluster_concentration',
        f'{suf}_cluster_entropy',
        f'{suf}_cluster_switches',
        f'{suf}_cluster_stability',
        f'main_{suf}_cluster_time_ratio',
        f'{suf}_mean_cluster_switch_days',
        f'{suf}_mean_query_len',
        f'{suf}_main_cluster_query_len_diff',
        f'days_since_last_{suf}',
        f'days_since_first_{suf}',
        f'last_{suf}_time',
        f'first_{suf}_time',
    )
)

In [ ]:
df_main = test_users_submission
for _, actions_aggs_df in actions_aggs.items():
    df_main = (
        df_main
        .join(actions_aggs_df, on='user_id', how='left')
    )
    
df_pd = df_main.to_pandas()

In [ ]:
from local_utils import *

df_pd = add_polynomial_features_pd(df_pd, num_cols, degree=3)

df_pd['predict'] = model.predict_proba(df_pd[cols])[:, 1]

In [ ]:
df_pd[['user_id', 'predict']]

In [ ]:
df_pd[['user_id', 'predict']].to_csv('res.csv', index=False)